In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

data = {
    'country': ['United States', 'Argentina', 'Australia', 'Brazil', 'Britain', 'Canada', 'Chile', 
                'China', 'Czech Rep', 'Denmark', 'Euro area', 'France', 'Germany', 'Italy', 'Spain', 
                'Hong Kong', 'Hungary', 'Indonesia', 'Japan', 'Malaysia', 'Mexico', 'New Zealand', 
                'Philippines', 'Poland', 'Russia', 'Singapore', 'South Africa', 'South Korea', 
                'Sweden', 'Switzerland', 'Taiwan', 'Thailand'],
    'implied_ppp': [1, 0.98, 1.18, 1.42, 1.28, 1.31, 496, 3.90, 22.0, 9.74, 0.99, 7.28, 2.01, 
                   1693, 156, 4.21, 157, 5787, 116, 1.78, 8.62, 1.42, 23.2, 2.32, 13.8, 1.30, 
                   3.82, 1181, 9.45, 2.48, 27.6, 21.7],
    'actual_rate': [1, 1.00, 1.98, 2.19, 1.43, 1.56, 601, 8.28, 39.0, 8.46, 0.88, 7.44, 2.22, 
                    2193, 189, 7.80, 303, 10855, 124, 3.80, 9.29, 2.47, 50.3, 4.03, 28.9, 1.81, 
                    8.13, 1325, 10.28, 1.73, 32.9, 45.5]
}

df = pd.DataFrame(data)


df = df[df['country'] != 'United States']


print(df.head())


     country  implied_ppp  actual_rate
1  Argentina         0.98         1.00
2  Australia         1.18         1.98
3     Brazil         1.42         2.19
4    Britain         1.28         1.43
5     Canada         1.31         1.56


In [2]:



# Регрессионный анализ
slope, intercept, r_value, p_value, std_err = stats.linregress(df['implied_ppp'], df['actual_rate'])

print(f"Коэффициент β₁ (intercept): {intercept}")
print(f"Коэффициент β₂ (slope): {slope}")
print(f"Коэффициент детерминации (R²): {r_value**2}")
print(f"p-значение: {p_value}")
print(f"Стандартная ошибка: {std_err}")



Коэффициент β₁ (intercept): -59.33156236991482
Коэффициент β₂ (slope): 1.8146556876190325
Коэффициент детерминации (R²): 0.9865994436288275
p-значение: 1.0304297651270698e-28
Стандартная ошибка: 0.03927228919586127


In [3]:

# Тест на β₁ = 0
t_stat_intercept = intercept / std_err
p_value_intercept = 2 * (1 - stats.t.cdf(abs(t_stat_intercept), df=len(df)-2))
print(f"Тест H₀: β₁ = 0, p-значение: {p_value_intercept}")



X = df['implied_ppp'].values.reshape(-1, 1)
X_with_const = np.hstack((np.ones((X.shape[0], 1)), X)) 
y = df['actual_rate'].values

# Оценка коэффициентов с помощью OLS
beta = np.linalg.inv(X_with_const.T @ X_with_const) @ X_with_const.T @ y
print(f"Коэффициенты из OLS: β₁ = {beta[0]}, β₂ = {beta[1]}")


Тест H₀: β₁ = 0, p-значение: 0.0
Коэффициенты из OLS: β₁ = -59.33156236991474, β₂ = 1.8146556876190325


In [4]:

# Остатки
residuals = y - X_with_const @ beta

# Оценка дисперсии ошибок
sigma_squared = np.sum(residuals**2) / (len(df) - 2)

# Ковариационная матрица оценок
cov_matrix = sigma_squared * np.linalg.inv(X_with_const.T @ X_with_const)

# Стандартная ошибка для β₂
se_beta2 = np.sqrt(cov_matrix[1, 1])

# Тест на β₂ = 1
t_stat_slope = (beta[1] - 1) / se_beta2
p_value_slope = 2 * (1 - stats.t.cdf(abs(t_stat_slope), df=len(df)-2))
print(f"Тест H₀: β₂ = 1, p-значение: {p_value_slope}")



Тест H₀: β₂ = 1, p-значение: 0.0


In [5]:

df_filtered = df[(df['implied_ppp'] < 100) & (df['actual_rate'] < 100)]

# Повторим регрессионный анализ для фильтрованных данных
slope_filtered, intercept_filtered, r_value_filtered, p_value_filtered, std_err_filtered = stats.linregress(
    df_filtered['implied_ppp'], df_filtered['actual_rate'])

print("\nРезультаты регрессии без экстремальных выбросов:")
print(f"Коэффициент β₁ (intercept): {intercept_filtered}")
print(f"Коэффициент β₂ (slope): {slope_filtered}")
print(f"Коэффициент детерминации (R²): {r_value_filtered**2}")
print(f"p-значение: {p_value_filtered}")

# Тест на β₁ = 0 и β₂ = 1 для фильтрованных данных
X_filtered = df_filtered['implied_ppp'].values.reshape(-1, 1)
X_filtered_with_const = np.hstack((np.ones((X_filtered.shape[0], 1)), X_filtered))
y_filtered = df_filtered['actual_rate'].values

# Оценка коэффициентов с помощью OLS для фильтрованных данных
beta_filtered = np.linalg.inv(X_filtered_with_const.T @ X_filtered_with_const) @ X_filtered_with_const.T @ y_filtered
residuals_filtered = y_filtered - X_filtered_with_const @ beta_filtered
sigma_squared_filtered = np.sum(residuals_filtered**2) / (len(df_filtered) - 2)
cov_matrix_filtered = sigma_squared_filtered * np.linalg.inv(X_filtered_with_const.T @ X_filtered_with_const)
se_beta1_filtered = np.sqrt(cov_matrix_filtered[0, 0])
se_beta2_filtered = np.sqrt(cov_matrix_filtered[1, 1])

# Тест на β₁ = 0
t_stat_intercept_filtered = beta_filtered[0] / se_beta1_filtered
p_value_intercept_filtered = 2 * (1 - stats.t.cdf(abs(t_stat_intercept_filtered), df=len(df_filtered)-2))
print(f"Тест H₀: β₁ = 0 (фильтрованные данные), p-значение: {p_value_intercept_filtered}")

# Тест на β₂ = 1
t_stat_slope_filtered = (beta_filtered[1] - 1) / se_beta2_filtered
p_value_slope_filtered = 2 * (1 - stats.t.cdf(abs(t_stat_slope_filtered), df=len(df_filtered)-2))
print(f"Тест H₀: β₂ = 1 (фильтрованные данные), p-значение: {p_value_slope_filtered}")



Результаты регрессии без экстремальных выбросов:
Коэффициент β₁ (intercept): -0.8272998137908907
Коэффициент β₂ (slope): 1.7333287043614787
Коэффициент детерминации (R²): 0.891053248539342
p-значение: 4.550017931361173e-12
Тест H₀: β₁ = 0 (фильтрованные данные), p-значение: 0.561847689183024
Тест H₀: β₂ = 1 (фильтрованные данные), p-значение: 1.0453512553132427e-05


In [6]:

# Совместный тест (F-тест) для фильтрованных данных
# Оценка с ограничениями (β₁ = 0, β₂ = 1)
y_restricted_filtered = df_filtered['actual_rate'] - df_filtered['implied_ppp']
SSR_restricted_filtered = np.sum(y_restricted_filtered**2)
SSR_unrestricted_filtered = np.sum(residuals_filtered**2)
q = 2  # Число ограничений
k = 2  # Число параметров
n_filtered = len(df_filtered)

F_stat_filtered = ((SSR_restricted_filtered - SSR_unrestricted_filtered) / q) / (SSR_unrestricted_filtered / (n_filtered - k))
p_value_F_filtered = 1 - stats.f.cdf(F_stat_filtered, q, n_filtered-k)

print(f"F-статистика для совместной гипотезы β₁ = 0 и β₂ = 1 (фильтрованные данные): {F_stat_filtered}")
print(f"p-значение для F-теста (фильтрованные данные): {p_value_F_filtered}")

F-статистика для совместной гипотезы β₁ = 0 и β₂ = 1 (фильтрованные данные): 25.261690870152414
p-значение для F-теста (фильтрованные данные): 2.0016630538632896e-06
